In [3]:
import numpy as np
import time

class Reader:
    def __init__(self, id, num_readers, alpha):
        self.id = id
        self.num_readers = num_readers
        self.alpha = alpha
        self.status = "LOCK"
        self.local_vertex = None
        self.neighbors = []
        self.cost = 0
    
    def set_local_vertex(self, num_neighbors):
        if num_neighbors <= self.alpha:
            self.local_vertex = {"weight": num_neighbors}
    
    def add_neighbor(self, neighbor_id):
        self.neighbors.append(neighbor_id)
    
    def remove_neighbor(self, neighbor_id):
        if neighbor_id in self.neighbors:
            self.neighbors.remove(neighbor_id)
    
    def compute_cost(self):
        if self.local_vertex:
            self.cost = self.local_vertex["weight"] * self.id
    
    def prepare_signals(self):
        signals = []
        if self.local_vertex:
            for neighbor_id in self.neighbors:
                signals.append((self.id, neighbor_id, "UPDATE", self.cost))
        return signals
    
    def send_signals(self, signals, all_readers):
        for signal in signals:
            sender_id, receiver_id, code, value = signal
            receiver = all_readers[receiver_id]
            if receiver.status == "ACTIVE":
                receiver.receive_signal(sender_id, code, value)
    
    def receive_signal(self, sender_id, code, value):
        if code == "UPDATE":
            self.cost = value
    
    def make_local_decision(self, all_readers):
        if not self.neighbors:  # No neighbors left
            self.status = "OFF"
            for neighbor_id in range(self.num_readers):
                if neighbor_id != self.id:
                    all_readers[neighbor_id].receive_signal(self.id, "DEACTIVATED", "N/A")
            return True
        
        max_cost_vertex_id = max(self.neighbors, key=lambda x: all_readers[x].cost)
        max_cost_vertex = all_readers[max_cost_vertex_id]
        
        if max_cost_vertex_id == self.id:
            self.status = "ACTIVE"
            for neighbor_id in self.neighbors:
                all_readers[neighbor_id].receive_signal(self.id, "ACTIVATED", "N/A")
            return True
        
        return False

def distributed_MWISBAII(num_readers, alpha, iterations=100):
    readers = [Reader(i, num_readers, alpha) for i in range(num_readers)]
    
    # Step-1: Initialize the graph
    for reader in readers:
        for neighbor_id in range(num_readers):
            if neighbor_id != reader.id:
                if np.random.rand() < 0.5:  # Randomly connect neighbors
                    reader.add_neighbor(neighbor_id)
        reader.set_local_vertex(len(reader.neighbors))
    
    for _ in range(iterations):
        # Step-2: Remove redundant vertices
        for reader in readers:
            if reader.status == "ACTIVE":
                reader.neighbors = []
            elif reader.status == "OFF":
                continue
            else:
                for neighbor_id in reader.neighbors.copy():
                    if readers[neighbor_id].status == "OFF":
                        reader.remove_neighbor(neighbor_id)
                if reader.status == "ACTIVE":
                    for neighbor_id in reader.neighbors.copy():
                        reader.remove_neighbor(neighbor_id)
        
        # Step-3: Compute cost
        for reader in readers:
            reader.compute_cost()
        
        # Step-4: Prepare signals
        all_signals = []
        for reader in readers:
            signals = reader.prepare_signals()
            all_signals.extend(signals)
        
        # Step-5: Send and receive signals
        for reader in readers:
            if reader.status == "ACTIVE":
                reader.status = "OPEN"
                reader.send_signals(all_signals, readers)
                time.sleep(0.1)  # Simulate waiting for signals
        
        # Step-6: Process received signals
        for reader in readers:
            if reader.status == "ACTIVE":
                reader.status = "LOCK"
                reader.make_local_decision(readers)
        
        # Step-7: Make a local decision
        active_readers = sum(1 for reader in readers if reader.status == "ACTIVE")
        if active_readers == 0:
            return "All readers turned off."
        elif active_readers == 1:
            return f"Reader {next(reader.id for reader in readers if reader.status == 'ACTIVE')} turned active."
        
    return "Algorithm didn't converge."

# Example usage
num_readers = 10
alpha = 0.5
print(distributed_MWISBAII(num_readers, alpha))


All readers turned off.


In [4]:
import numpy as np
import time

class Reader:
    def __init__(self, id, num_readers, alpha, threshold):
        self.id = id
        self.num_readers = num_readers
        self.alpha = alpha
        self.threshold = threshold
        self.status = "LOCK"
        self.local_vertex = None
        self.neighbors = []
        self.cost = 0
    
    def set_local_vertex(self, num_neighbors):
        if num_neighbors <= self.alpha:
            self.local_vertex = {"weight": num_neighbors}
    
    def add_neighbor(self, neighbor_id):
        self.neighbors.append(neighbor_id)
    
    def remove_neighbor(self, neighbor_id):
        if neighbor_id in self.neighbors:
            self.neighbors.remove(neighbor_id)
    
    def compute_cost(self):
        if self.local_vertex:
            self.cost = self.local_vertex["weight"] * self.id
    
    def prepare_signals(self):
        signals = []
        if self.local_vertex:
            for neighbor_id in self.neighbors:
                signals.append((self.id, neighbor_id, "UPDATE", self.cost))
        return signals
    
    def send_signals(self, signals, all_readers):
        for signal in signals:
            sender_id, receiver_id, code, value = signal
            receiver = all_readers[receiver_id]
            if receiver.status == "ACTIVE":
                receiver.receive_signal(sender_id, code, value)
    
    def receive_signal(self, sender_id, code, value):
        if code == "UPDATE":
            self.cost = value
    
    def make_local_decision(self, all_readers):
        if not self.neighbors:  # No neighbors left
            self.status = "OFF"
            for neighbor_id in range(self.num_readers):
                if neighbor_id != self.id:
                    all_readers[neighbor_id].receive_signal(self.id, "DEACTIVATED", "N/A")
            return True
        
        max_cost_vertex_id = max(self.neighbors, key=lambda x: all_readers[x].cost)
        max_cost_vertex = all_readers[max_cost_vertex_id]
        
        if max_cost_vertex_id == self.id:
            self.status = "ACTIVE"
            for neighbor_id in self.neighbors:
                all_readers[neighbor_id].receive_signal(self.id, "ACTIVATED", "N/A")
            return True
        
        return False

def distributed_MWISBAII(num_readers, alpha, threshold, iterations=100):
    readers = [Reader(i, num_readers, alpha, threshold) for i in range(num_readers)]
    
    for _ in range(iterations):
        # Step-1: Initialize the graph
        for reader in readers:
            for neighbor_id in range(num_readers):
                if neighbor_id != reader.id:
                    if np.random.rand() < 0.5:  # Randomly connect neighbors
                        reader.add_neighbor(neighbor_id)
            reader.set_local_vertex(len(reader.neighbors))
        
        # Step-2: Remove redundant vertices
        for reader in readers:
            if reader.status == "ACTIVE":
                reader.neighbors = []
            elif reader.status == "OFF":
                continue
            else:
                for neighbor_id in reader.neighbors.copy():
                    if readers[neighbor_id].status == "OFF":
                        reader.remove_neighbor(neighbor_id)
                if reader.status == "ACTIVE":
                    for neighbor_id in reader.neighbors.copy():
                        reader.remove_neighbor(neighbor_id)
        
        # Step-3: Compute cost
        for reader in readers:
            reader.compute_cost()
        
        # Step-4: Prepare signals
        all_signals = []
        for reader in readers:
            signals = reader.prepare_signals()
            all_signals.extend(signals)
        
        # Step-5: Send and receive signals
        for reader in readers:
            if reader.status == "ACTIVE":
                reader.status = "OPEN"
                reader.send_signals(all_signals, readers)
                time.sleep(0.1)  # Simulate waiting for signals
        
        # Step-6: Process received signals
        for reader in readers:
            if reader.status == "ACTIVE":
                reader.status = "LOCK"
                reader.make_local_decision(readers)
        
        # Step-7: Make a local decision
        active_readers = sum(1 for reader in readers if reader.status == "ACTIVE")
        if active_readers == 0:
            return "All readers turned off."
        elif active_readers == 1:
            return f"Reader {next(reader.id for reader in readers if reader.status == 'ACTIVE')} turned active."
        
    return "Algorithm didn't converge."

# Example usage
num_readers = 10
alpha = 0.5
threshold = 10  # Set threshold for each reader
print(distributed_MWISBAII(num_readers, alpha, threshold))


All readers turned off.


In [5]:
import numpy as np
import time

class Reader:
    def __init__(self, id, num_readers, alpha, threshold):
        self.id = id
        self.num_readers = num_readers
        self.alpha = alpha
        self.threshold = threshold
        self.status = "LOCK"
        self.local_vertex = None
        self.neighbors = []
        self.cost = 0
    
    def set_local_vertex(self, num_neighbors):
        if num_neighbors <= self.alpha:
            self.local_vertex = {"weight": num_neighbors}
    
    def add_neighbor(self, neighbor_id):
        self.neighbors.append(neighbor_id)
    
    def remove_neighbor(self, neighbor_id):
        if neighbor_id in self.neighbors:
            self.neighbors.remove(neighbor_id)
    
    def compute_cost(self):
        if self.local_vertex:
            self.cost = self.local_vertex["weight"] * self.id
    
    def prepare_signals(self):
        signals = []
        if self.local_vertex:
            for neighbor_id in self.neighbors:
                signals.append((self.id, neighbor_id, "UPDATE", self.cost))
        return signals
    
    def send_signals(self, signals, all_readers):
        for signal in signals:
            sender_id, receiver_id, code, value = signal
            receiver = all_readers[receiver_id]
            if receiver.status == "ACTIVE":
                receiver.receive_signal(sender_id, code, value)
    
    def receive_signal(self, sender_id, code, value):
        if code == "UPDATE":
            self.cost = value
        elif code == "ACTIVATE":
            self.status = "ACTIVE"
        elif code == "DEACTIVATED":
            self.status = "OFF"
    
    def make_local_decision(self, all_readers):
        if not self.neighbors:  # No neighbors left
            self.status = "OFF"
            for neighbor_id in range(self.num_readers):
                if neighbor_id != self.id:
                    all_readers[neighbor_id].receive_signal(self.id, "DEACTIVATED", "N/A")
            return True
        
        max_cost_vertex_id = max(self.neighbors, key=lambda x: all_readers[x].cost)
        max_cost_vertex = all_readers[max_cost_vertex_id]
        
        if max_cost_vertex_id == self.id:
            self.status = "ACTIVE"
            for neighbor_id in self.neighbors:
                all_readers[neighbor_id].receive_signal(self.id, "ACTIVATE", "N/A")
            return True
        
        return False

def distributed_MWISBAII(num_readers, alpha, threshold, iterations=100):
    readers = [Reader(i, num_readers, alpha, threshold) for i in range(num_readers)]
    
    for _ in range(iterations):
        # Step-1: Initialize the graph
        for reader in readers:
            for neighbor_id in range(num_readers):
                if neighbor_id != reader.id:
                    if np.random.rand() < 0.5:  # Randomly connect neighbors
                        reader.add_neighbor(neighbor_id)
            reader.set_local_vertex(len(reader.neighbors))
        
        # Step-2: Remove redundant vertices
        for reader in readers:
            if reader.status == "ACTIVE":
                reader.neighbors = []
            elif reader.status == "OFF":
                continue
            else:
                for neighbor_id in reader.neighbors.copy():
                    if readers[neighbor_id].status == "OFF":
                        reader.remove_neighbor(neighbor_id)
                if reader.status == "ACTIVE":
                    for neighbor_id in reader.neighbors.copy():
                        reader.remove_neighbor(neighbor_id)
        
        # Step-3: Compute cost
        for reader in readers:
            reader.compute_cost()
        
        # Step-4: Prepare signals
        all_signals = []
        for reader in readers:
            signals = reader.prepare_signals()
            all_signals.extend(signals)
        
        # Step-5: Send and receive signals
        for reader in readers:
            if reader.status == "ACTIVE":
                reader.status = "OPEN"
                reader.send_signals(all_signals, readers)
                time.sleep(0.1)  # Simulate waiting for signals
        
        # Step-6: Process received signals
        for reader in readers:
            if reader.status == "ACTIVE":
                reader.status = "LOCK"
                reader.make_local_decision(readers)
        
        # Step-7: Make a local decision
        active_readers = sum(1 for reader in readers if reader.status == "ACTIVE")
        if active_readers == 0:
            return "All readers turned off."
        elif active_readers == 1:
            return f"Reader {next(reader.id for reader in readers if reader.status == 'ACTIVE')} turned active."
        
    return "Algorithm didn't converge."

# Example usage
num_readers = 10
alpha = 0.5
threshold = 10  # Set threshold for each reader
print(distributed_MWISBAII(num_readers, alpha, threshold))


All readers turned off.
